In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt




import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/kaggle/input/thyroid-cancer-risk-dataset/thyroid_cancer_risk_data.csv")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.corr

In [ ]:
df.isnull().sum()

In [ ]:
df["Family_History"] = df["Family_History"].map({"Yes": 1, "No": 0})  # Adjust based on actual values
df["Thyroid_Cancer_Risk"] = pd.to_numeric(df["Thyroid_Cancer_Risk"], errors="coerce")

In [ ]:
# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# 1. Age Distribution
plt.figure(figsize=(8, 5))
sns.histplot(df["Age"], kde=True, bins=30, color="skyblue")
plt.title("Age Distribution of Patients")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.show()

# 2. Boxplot of TSH, T3, and T4 Levels
plt.figure(figsize=(12, 5))
sns.boxplot(data=df[["TSH_Level", "T3_Level", "T4_Level"]], palette=["darkblue", "darkgreen", "darkred"])
plt.title("Boxplot of Thyroid Hormone Levels")
plt.ylabel("Hormone Levels")
plt.show()

# 3. Gender vs Diagnosis
plt.figure(figsize=(6, 4))
sns.countplot(x="Gender", hue="Diagnosis", data=df, palette="coolwarm")
plt.title("Thyroid Cancer Diagnosis by Gender")
plt.xlabel("Gender")
plt.ylabel("Count")
plt.show()

# 4. Country-wise Distribution
plt.figure(figsize=(12, 5))
df["Country"].value_counts().plot(kind="bar", color="skyblue")
plt.title("Patient Distribution by Country")
plt.xlabel("Country")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.show()

# 5. Family History vs Thyroid Cancer Risk
plt.figure(figsize=(6, 4))
sns.barplot(x="Family_History", y="Thyroid_Cancer_Risk", data=df, palette="Blues")
plt.title("Family History vs Cancer Risk")
plt.ylabel("Cancer Risk %")
plt.show()

# 6. Radiation Exposure & Cancer Diagnosis
plt.figure(figsize=(6, 4))
sns.countplot(x="Radiation_Exposure", hue="Diagnosis", data=df, palette="coolwarm")
plt.title("Radiation Exposure and Thyroid Cancer Diagnosis")
plt.xlabel("Radiation Exposure")
plt.ylabel("Count")
plt.show()

# 7. Smoking & Obesity vs Cancer Risk
plt.figure(figsize=(8, 5))
sns.barplot(x="Smoking", y="Thyroid_Cancer_Risk", hue="Obesity", data=df, palette="Blues")
plt.title("Smoking, Obesity, and Cancer Risk")
plt.xlabel("Smoking Status")
plt.ylabel("Cancer Risk %")
plt.show()


# 8. Correlation Heatmap
plt.figure(figsize=(10, 6))
corr = df[["Age", "TSH_Level", "T3_Level", "T4_Level", "Nodule_Size", "Thyroid_Cancer_Risk"]].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()

# 9. Pairplot for Key Features
sns.pairplot(df, vars=["Age", "Nodule_Size", "TSH_Level", "T3_Level", "T4_Level"], hue="Diagnosis", palette=["darkblue", "darkred"], diag_kind="kde")
plt.show()

In [ ]:
df_categorical = df.select_dtypes(include='object')
label_encoders = {}
for column in df_categorical.columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])  
    label_encoders[column] = le  
print(df.head())

In [ ]:
from sklearn.preprocessing import StandardScaler
x = df.drop(columns=["Patient_ID", "Diagnosis"])
y = df["Diagnosis"]
x_scaled = StandardScaler().fit_transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

In [ ]:
for col in df.columns:
    if df[col].dtype == "object":
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].median(), inplace=True)

In [ ]:
categorical_cols = ['Gender', 'Country', 'Ethnicity', 'Family_History', 
                    'Radiation_Exposure', 'Iodine_Deficiency', 'Smoking', 
                    'Obesity', 'Diabetes']
encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])

In [ ]:
scaler = StandardScaler()
num_cols = ['Age', 'TSH_Level', 'T3_Level', 'T4_Level', 'Nodule_Size']
df[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
for i, col in enumerate(['TSH_Level', 'T3_Level', 'T4_Level'], 1):
    plt.subplot(1, 3, i)
    sns.boxplot(x=y, y=df[col])
    plt.title(f"{col} vs Diagnosis")
plt.show()

In [ ]:
df['Thyroid_Cancer_Risk'].fillna(df['Thyroid_Cancer_Risk'].median(), inplace=True)

In [ ]:
print(df.isnull().sum())

In [ ]:
imputer = SimpleImputer(strategy='mean')  # or 'median', 'most_frequent'
x_train = imputer.fit_transform(x_train)
x_test = imputer.transform(x_test)

In [ ]:
print("Original X columns:", df.drop(columns=["Patient_ID", "Diagnosis"]).shape[1])
print("Scaled X columns:", x_scaled.shape[1])

In [ ]:
imputer = SimpleImputer(strategy="mean")  # or "median", "most_frequent"
X_imputed = imputer.fit_transform(df.drop(columns=["Patient_ID", "Diagnosis"]))

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

In [ ]:
feature_names = df.drop(columns=["Patient_ID", "Diagnosis"]).columns[:x_train.shape[1]]
x_train = pd.DataFrame(x_train, columns=feature_names)
x_test = pd.DataFrame(x_test, columns=feature_names)

In [ ]:
models_params = {
    'RandomForest': (RandomForestClassifier(random_state=42), {
        'n_estimators': [50, 100],
        'max_depth': [None, 10],
    }),
    'LogisticRegression': (LogisticRegression(random_state=42, max_iter=500), {
        'C': [0.01, 1, 100],
    }),
    'XGBoost': (XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'), {
        'n_estimators': [50, 100],
        'learning_rate': [0.01, 0.1],
    })
}

results = {}
for name, (model, param_dist) in models_params.items():
    print(f"Training {name}...")
    
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=3,  
        scoring='accuracy',
        cv=3,
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    search.fit(x_train, y_train)
    
    best_model = search.best_estimator_
    y_pred = best_model.predict(x_test)
    
    results[name] = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {results[name]:.4f}")

# Visualization
plt.figure(figsize=(6, 4))
plt.bar(results.keys(), results.values(), color=['blue', 'green', 'orange'])
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.title("Model Accuracy Comparison")
plt.ylim(0, 1)
plt.show()

In [ ]:
!pip install catboost lightgbm


In [ ]:



# Update the models_params dictionary to include SVM
models_params = {
    'RandomForest': (RandomForestClassifier(random_state=42), {
        'n_estimators': [50, 100],
        'max_depth': [None, 10],
    }),
    'LogisticRegression': (LogisticRegression(random_state=42, max_iter=500), {
        'C': [0.01, 1, 100],
    }),
    'XGBoost': (XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'), {
        'n_estimators': [50, 100],
        'learning_rate': [0.01, 0.1],
    })
 


}

# Training and evaluation loop
results = {}
for name, (model, param_dist) in models_params.items():
    print(f"Training {name}...")

    # RandomizedSearchCV for hyperparameter tuning
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=5,  
        scoring='accuracy',
        cv=3,
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    search.fit(x_train, y_train)

    # Get the best model and predict
    best_model = search.best_estimator_
    y_pred = best_model.predict(x_test)

    # Store accuracy
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    print(f"{name} best params: {search.best_params_}")
    print(f"{name} Accuracy: {accuracy:.4f}\n")



In [ ]:
# Example Voting Classifier (Soft Voting)
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier

voting_clf = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(random_state=42)),
        ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
       
    ],
    voting='soft'
)

# Train and evaluate Voting Classifier
voting_clf.fit(x_train, y_train)
y_pred_vote = voting_clf.predict(x_test)
results['VotingClassifier'] = accuracy_score(y_test, y_pred_vote)
print("Voting Classifier Accuracy:", results['VotingClassifier'])

# Visualization
plt.figure(figsize=(10, 6))  # Increased figure size for better readability
plt.bar(results.keys(), results.values(), color=['blue', 'green', 'orange', 'purple', 'red', 'cyan', 'magenta', 'yellow', 'pink'])
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.title("Model Accuracy Comparison")
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Print final accuracy scores
print("\n=== Model Accuracy Scores ===")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy:.4f}")

In [ ]:
print("\n=== Model Accuracy Scores ===")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy:.4f}")

# Optional: Bar plot for model comparison
plt.figure(figsize=(8, 5))
plt.bar(results.keys(), results.values(), color=['blue', 'green', 'orange', 'purple', 'red', 'cyan', 'magenta'])
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.title("Model Accuracy Comparison")
plt.ylim(0, 1)
plt.show()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

results = {}

for name, (model, param_dist) in models_params.items():
    print(f"Training {name}...")
    rs = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=5, cv=3, random_state=42, n_jobs=-1)
    rs.fit(x_train, y_train)
    
    best_model = rs.best_estimator_
    y_pred = best_model.predict(x_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    
    print(f"{name} best params: {rs.best_params_}")
    print(f"{name} Accuracy: {accuracy:.4f}\n")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Dictionary to store predictions for each model
model_predictions = {}

# Collect predictions for RandomForest, LogisticRegression, and XGBoost
for name, (model, param_dist) in models_params.items():
    rs = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=5, cv=3, random_state=42, n_jobs=-1)
    rs.fit(x_train, y_train)
    best_model = rs.best_estimator_
    model_predictions[name] = best_model.predict(x_test)

# Collect predictions for VotingClassifier
model_predictions['VotingClassifier'] = voting_clf.predict(x_test)

# Plot confusion matrices for all models
plt.figure(figsize=(12, 10))
for i, (name, y_pred) in enumerate(model_predictions.items(), 1):
    cm = confusion_matrix(y_test, y_pred)
    plt.subplot(2, 2, i)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix: {name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
plt.tight_layout()
plt.show()